[View in Colaboratory](https://colab.research.google.com/github/fatwiz/fastai/blob/master/lesson1_quick_dogs_v_cats.ipynb)

#Quick Dogs v Cats

## Setup VM for use with fast.ai

### Check VM Status

Check RAM allocation

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - done
  Stored in directory: /root/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 12.6 GB  | Proc size: 139.7 MB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB


Uncomment line to restart Jupyter Kernel if necessary

In [0]:
#!pkill -9 -f ipykernel_launcher

Python version

In [2]:
!python -V

Python 3.6.3


### Import libraries to use fast.ai

In [3]:
!pip install fastai

    100% |████████████████████████████████| 122kB 3.9MB/s 
    100% |████████████████████████████████| 3.6MB 9.0MB/s 
    100% |████████████████████████████████| 51kB 19.0MB/s 
    100% |████████████████████████████████| 51kB 18.2MB/s 
    100% |████████████████████████████████| 1.5MB 13.5MB/s 
    100% |████████████████████████████████| 61kB 18.0MB/s 
    100% |████████████████████████████████| 496.4MB 31kB/s 
    100% |████████████████████████████████| 2.2MB 8.8MB/s 
    100% |████████████████████████████████| 112kB 19.0MB/s 
    100% |████████████████████████████████| 184kB 18.7MB/s 
    100% |████████████████████████████████| 71kB 14.0MB/s 
    100% |████████████████████████████████| 901kB 2.8MB/s 
    100% |████████████████████████████████| 112kB 19.0MB/s 
    100% |████████████████████████████████| 11.6MB 2.7MB/s 
    100% |████████████████████████████████| 92kB 22.9MB/s 
    100% |████████████████████████████████| 81kB 19.2MB/s 
    100% |████████████████████████████████| 15.8MB

##Dogs versus Cats

###Download Dataset

In [4]:
!mkdir data && wget http://files.fast.ai/data/dogscats.zip && unzip -q dogscats.zip -d data/

--2018-08-27 08:48:36--  http://files.fast.ai/data/dogscats.zip
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857214334 (818M) [application/zip]
Saving to: ‘dogscats.zip’

dogscats.zip        100%[===================>] 817.50M   103MB/s    in 8.1s    

2018-08-27 08:48:44 (101 MB/s) - ‘dogscats.zip’ saved [857214334/857214334]



###Finally the actual work

In [0]:
from fastai.conv_learner import *
PATH = "data/dogscats"
sz=224; bs=64

In [6]:
tfms = tfms_from_model(resnet50, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)
learn = ConvLearner.pretrained(resnet50, data)
%time learn.fit(1e-2, 3, cycle_len=1)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:01<00:00, 64691963.21it/s]


epoch      trn_loss   val_loss   accuracy   
    0      0.05058    0.027811   0.9895    
    1      0.033619   0.025975   0.991     
    2      0.034842   0.026121   0.9895    

CPU times: user 18min 2s, sys: 3min, total: 21min 3s
Wall time: 13min 37s


[array([0.02612]), 0.9895]

In [7]:
learn.unfreeze()
learn.bn_freeze(True)
%time learn.fit([1e-5,1e-4,1e-2], 1, cycle_len=1)

epoch      trn_loss   val_loss   accuracy   
    0      0.021984   0.023438   0.991     

CPU times: user 10min 10s, sys: 1min 57s, total: 12min 7s
Wall time: 9min 34s


[array([0.02344]), 0.991]

In [13]:
%time log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

CPU times: user 2min 11s, sys: 19.3 s, total: 2min 31s
Wall time: 1min 32s


(0.994, 0.01659466730582006)